In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandlesA4").getOrCreate()

22/04/01 02:24:32 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/04/01 02:24:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-100c6ddf-5e48-438b-90c5-2bd2e4b7d3cb;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 600ms :: artifacts dl 10ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:

kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "orders")\
  .option("group.id", "orders-group-rm")\
  .load()

In [5]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# echoOnconsole =kafkaDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [6]:
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
# echoOnconsole =kafkaDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data


In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType,IntegerType

schema = StructType( [
    StructField("Order_id", IntegerType(), True),
    StructField("Item_id", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", IntegerType(),  True),
    StructField("State",StringType(), True),
    StructField("timestamp", LongType(),  True)
])

In [8]:
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- Order_id: integer (nullable = true)
 |    |-- Item_id: string (nullable = true)
 |    |-- Quantity: integer (nullable = true)
 |    |-- Price: integer (nullable = true)
 |    |-- State: string (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [9]:
stockTickDf = jsonDf.select (F.col("value.*")) # extract all sub fields as schema
stockTickDf.printSchema()

root
 |-- Order_id: integer (nullable = true)
 |-- Item_id: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- timestamp: long (nullable = true)



In [12]:
echoOnconsole =stockTickDf\
                .writeStream\
                .outputMode("update")\
                .format("console")\
                .option("truncate", False)\
                .start() # start the query. spark will subscribe for data

22/04/01 00:39:52 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-09fc42c6-eb03-4a42-a911-f7933d382db1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+-------+--------+-----+-----+---------+
|Order_id|Item_id|Quantity|Price|State|timestamp|
+--------+-------+--------+-----+-----+---------+
+--------+-------+--------+-----+-----+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-------+--------+-----+-----+-------------+
|Order_id|Item_id|Quantity|Price|State|timestamp    |
+--------+-------+--------+-----+-----+-------------+
|65020   |58     |2       |45   |KY   |1648753793624|
|51105   |2      |7       |8    |KY   |1648753793624|
|25489   |16     |6       |27   |KY   |1648753793624|
|35588   |13     |4       |3    |KY   |1648753793624|
|44610   |33     |7       |18   |KY   |1648753793624|
|27531   |15     |5       |41   |KY   |1648753793624|
|11973   |28     |8       |20   |KY   |1648753793624|
|40702   |11     |2       |30   |KY   |1648753793624|
|64118   |6      |7       |6    |KY   |1648753793624|
+--------+-------+--------+-----+-----+-------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-------+--------+-----+-----+-------------+
|Order_id|Item_id|Quantity|Price|State|timestamp    |
+--------+-------+--------+-----+-----+-------------+
|21695   |62     |5       |40   |ME   |1648753798628|
|3390    |54     |5       |7    |ME   |1648753798628|
|58702   |61     |10      |45   |ME   |1648753798628|
|98556   |98     |2       |33   |ME   |1648753798628|
|28971   |33     |3       |20   |ME   |1648753798628|
|86792   |90     |10      |13   |ME   |1648753798628|
|25677   |21     |5       |37   |ME   |1648753798628|
|24676   |93     |6       |50   |ME   |1648753798628|
+--------+-------+--------+-----+-----+-------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------+-------+--------+-----+-----+-------------+
|Order_id|Item_id|Quantity|Price|State|timestamp    |
+--------+-------+--------+-----+-----+----------

In [10]:
ordersDf = stockTickDf.withColumn("Amount", F.col("Price") * F.col("Quantity"))
ordersDf.printSchema()

root
 |-- Order_id: integer (nullable = true)
 |-- Item_id: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- Amount: integer (nullable = true)



In [11]:
orderTickDf = ordersDf\
                .withColumn("timestampTemp", (F.col("timestamp") / 1000).cast("timestamp"))\
                .withColumn("trade_time", F.date_trunc("minute", F.col("timestampTemp")))\
                .drop("timestamp")\
                .drop("timestampTemp")\
                .withColumnRenamed("trade_time", "timestamp")

In [15]:
echoOnconsole = orderTickDf\
                .writeStream\
                .outputMode("update")\
                .format("console")\
                .option("truncate", False)\
                .start() # start the query. spark will subscribe for data

22/04/01 00:41:25 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4cbdafc4-8112-43d9-a1a0-4ac96bbb0711. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+-------+--------+-----+-----+------+---------+
|Order_id|Item_id|Quantity|Price|State|Amount|timestamp|
+--------+-------+--------+-----+-----+------+---------+
+--------+-------+--------+-----+-----+------+---------+

-------------------------------------------
Batch: 20
-------------------------------------------
+--------+-------+--------+-----+-----+-------------+
|Order_id|Item_id|Quantity|Price|State|timestamp    |
+--------+-------+--------+-----+-----+-------------+
|76028   |40     |10      |12   |AZ   |1648753888715|
|11739   |81     |1       |45   |AZ   |1648753888715|
|71628   |89     |1       |41   |AZ   |1648753888715|
+--------+-------+--------+-----+-----+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-------+--------+-----+-----+------+-------------------+
|Order_id|Item_id|Quantity|Price|State|Amount|timestamp          |
+--------+-------+--------+-----+-----+------+-------------------+
|76028   |40     |10      |12   |AZ   |120   |2022-04-01 00:41:00|
|11739   |81     |1       |45   |AZ   |45    |2022-04-01 00:41:00|
|71628   |89     |1       |41   |AZ   |41    |2022-04-01 00:41:00|
+--------+-------+--------+-----+-----+------+-------------------+



In [12]:
order5minDf = orderTickDf.withWatermark("timestamp","1 minute").groupBy("State",F.window("timestamp","5 minutes")).agg(F.sum("Amount").alias("Total_amount"))
                    
order5minDf.printSchema()

root
 |-- State: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- Total_amount: long (nullable = true)



In [13]:
## Converting back to kafka writable format
order5minDfKafka = order5minDf\
                            .selectExpr("to_json(struct(*)) AS value")


In [18]:
echoOnconsole =order5minDfKafka\
                .writeStream\
                .outputMode("update")\
                .format("console")\
                .option("truncate", False)\
                .start() # start the query. spark will subscribe for data

22/04/01 00:42:09 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bfda4eb0-15c2-4125-8a9e-2f34c7b1782a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 9
-------------------------------------------
+--------+-------+--------+-----+-----+------+-------------------+
|Order_id|Item_id|Quantity|Price|State|Amount|timestamp          |
+--------+-------+--------+-----+-----+------+-------------------+
|14653   |64     |4       |31   |ME   |124   |2022-04-01 00:42:00|
|79663   |55     |8       |35   |ME   |280   |2022-04-01 00:42:00|
|34156   |65     |4       |18   |ME   |72    |2022-04-01 00:42:00|
|22682   |59     |4       |25   |ME   |100   |2022-04-01 00:42:00|
|63108   |48     |9       |42   |ME   |378   |2022-04-01 00:42:00|
|50786   |79     |4       |7    |ME   |28    |2022-04-01 00:42:00|
|34040   |22     |3       |7    |ME   |21    |2022-04-01 00:42:00|
+--------+-------+--------+-----+-----+------+-------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-----------------------------------

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|{"State":"ME","window":{"start":"2022-04-01T00:40:00.000+05:30","end":"2022-04-01T00:45:00.000+05:30"},"Total_amount":834}|
+--------------------------------------------------------------------------------------------------------------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 30
-------------------------------------------
+--------+-------+--------+-----+-----+----------

In [14]:
order5minDfKafka \
            .writeStream\
             .format("kafka")\
            .outputMode("update")\
             .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("topic", "orders-5min")\
            .option("checkpointLocation", "file:///tmp/spark3")\
            .start()

22/04/01 02:26:08 WARN HDFSBackedStateStoreProvider: The state for version 459 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
22/04/01 02:26:09 WARN HDFSBackedStateStoreProvider: The state for version 459 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
22/04/01 02:26:09 WARN HDFSBackedStateStoreProvider: The state for version 459 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
22/04/01 02:26:09 WARN HDFSBackedStateStoreProvider: The state for version 459 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
